## Para fazer:

perceber cada linha de codigo ao mais infimo detalhe

perceber mesmo o que é um tensor

tiar collum do cancro do pulmão. fai servir para delta e weight delta ou para aume tat o erro na função de sum of scare residuals

tornar todos os nomes com letras pequenas e não existir espaos nos nomes

ver codigo exemplo em: https://github.com/IEEE-NOVA-SB/Easy_PyTorch_Models

In [1]:
import torch #create tensors to raw data and weights
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F #gives activation function  
from torch.optim import SGD #stochastic gradient descend
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'data.csv')
pd.options.display.max_columns = None

In [3]:
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'YES': 1, 'NO': 0})
df['GENDER'] = df['GENDER'].map({'M': 1, 'F': 0})
collumsToChange = ["SMOKING", "YELLOW_FINGERS","ANXIETY","PEER_PRESSURE","CHRONIC DISEASE","FATIGUE","ALLERGY","WHEEZING","ALCOHOL CONSUMING","COUGHING","SHORTNESS OF BREATH","SWALLOWING DIFFICULTY","CHEST PAIN"] 
for collum in collumsToChange:
    df[collum] = df[collum].map({2: 1, 1: 0})
df = df.astype('int64')

In [4]:
lungCancerTrueValues = df["LUNG_CANCER"].to_numpy()

df = df.drop('LUNG_CANCER', axis=1)

In [5]:
df_arr = df.to_records(index=False)
arr = []
for i in df_arr:
    for j in i:
        arr.append(j)
 

dataArray = np.array(arr) 

In [6]:
#Getting training dataset:
trainingArray = []
testArray = []
trainingArraySize = int(len(dataArray)*0.8)

trainingArray = dataArray[0:trainingArraySize]
testArray = dataArray[trainingArraySize:]

print(trainingArray)

[ 1 69  0 ...  1 67  0]


## Porque considerei que os homens é 1 e mulheres é 0?

Porque os homens têm um metabolismo mais alto, mair chance de trabalhar em ambientes poluidos(cavernas), entre outros.

Assim, na parte do género tenta-se compensar estas variaveis não quantificaveis

In [7]:
trainData=torch.tensor(trainingArray,dtype=torch.float32)
testData=torch.tensor(testArray,dtype=torch.float32)
target = torch.tensor(lungCancerTrueValues,dtype=torch.float32)

In [8]:
batch_size_num = 64

train_dataloader = DataLoader(trainData, batch_size=batch_size_num, shuffle=True)
test_dataloader = DataLoader(testData, batch_size=batch_size_num, shuffle=True)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(15, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=15, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):#usar target
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")